### 10 Academy: Artificial Intelligence Mastery
#### Week 3 Challenge
##### A/B_Hypothesis_Testing
Ethel Cherotaw

In [1]:
import sys
import numpy as np
import os
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score

data_dir=data_dir = r'E:\2017.Study\Tenx\Week-3\Data\data'
src_dir = r'E:\2017.Study\Tenx\Week-3\Insurance\W3.Insurance-Planning.AIM2\src'


sys.path.append(src_dir)
sys.path.append(data_dir)
from Modeling_utils import InsuranceDataUtils

In [2]:
csv_file_path = r'E:\2017.Study\Tenx\Week-3\Data\data\cleaned_data.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
df.head()


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


##### Data Processing 
Even though the data is already cleaned, let's double-check for any remaining missing values.

In [3]:
missing_values = df.isnull().sum()

# Display columns with missing values and the count
missing_values = missing_values[missing_values > 0]
print(missing_values)

Series([], dtype: int64)


 Series([ ], dtype: int64): indicates that there are no missing values in our dataset. The data is already cleaned on task-1 and there are no missing values to handle.


##### Feature Engineering

Feature engineering: is the process of transforming raw data into relevant information for use by machine learning models. In other words, feature engineering is the process of creating predictive model features. A feature—also called a dimension—is an input variable used to generate model predictions. Because model performance largely rests on the quality of data used during training, feature engineering is a crucial preprocessing technique that requires selecting the most relevant aspects of raw training data for both the predictive task and model type under consideration.[1]

New feature 

1. Extracting information from dates, such as the age of the vehicle from RegistrationYear.
2. Binning SumInsured and cubiccapacity

Converting continuous variables into categorical bins (e.g., "Low," "Medium," "High"). This can help models to understand different risk categories more easily.

3. Extracting Information from TransactionMonth:

features such as:

Month: Helps identify seasonality.

Quarter: Groups months into quarters.

Year: Can be useful if the data spans multiple years.

4.  Log-Transforming Skewed Features:

Features like SumInsured and CalculatedPremiumPerTerm can be highly skewed. Applying a log transformation can normalize these distributions.

In [4]:
insurance_utils = InsuranceDataUtils(df)

# Run preprocessing steps
df_processed = insurance_utils.preprocess()

# Display the first few rows of the processed DataFrame
df_processed.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,PostalCode,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,...,Section_Motor Comprehensive,Section_Optional Extended Covers,Section_Standalone passenger liability,"Section_Third party or third party, fire and theft only",Product_Mobility Commercial Cover: Monthly,Product_Mobility Metered Taxis: Monthly,Product_Standalone Passenger Liability,Season_Spring,Season_Summer,Season_Winter
0,0.638791,0.920659,2015-03-01,-0.589316,-1.908809,MERCEDES-BENZ,E 240,6.645499,0.293942,1.69074,...,True,False,False,False,False,True,False,True,False,False
1,0.638791,0.920659,2015-05-01,-0.589316,-1.908809,MERCEDES-BENZ,E 240,6.645499,0.293942,1.69074,...,True,False,False,False,False,True,False,True,False,False
2,0.638791,0.920659,2015-07-01,-0.589316,-1.908809,MERCEDES-BENZ,E 240,6.645499,0.293942,1.69074,...,True,False,False,False,False,True,False,False,True,False
3,0.638886,0.920659,2015-05-01,-0.589316,-1.908809,MERCEDES-BENZ,E 240,6.645499,0.293942,1.69074,...,True,False,False,False,False,True,False,True,False,False
4,0.638886,0.920659,2015-07-01,-0.589316,-1.908809,MERCEDES-BENZ,E 240,6.645499,0.293942,1.69074,...,True,False,False,False,False,True,False,False,True,False


In [ ]:
utils = InsuranceDataUtils(df)
df_processed = utils.preprocess()

# Split the data
X_train, X_test, y_train, y_test = utils.train_test_split()

# Build and train models
models = utils.build_models(X_train, y_train)

# Evaluate models
results = utils.evaluate_models(models, X_test, y_test)
for model_name, metrics in results.items():
    print(f"{model_name} Metrics: {metrics}")